In [ ]:
import os
import numpy as np
import pandas as pd
import gc

# To ensemble we used submissions from 3 public notebooks + 3 of our own models:
* LB: 0.0232 - https://www.kaggle.com/code/negoto/h-m-framework-for-partitioned-validation
* LB: 0.0231 - https://www.kaggle.com/code/ebn7amdi/trending/notebook?scriptVersionId=90980162
* LB: 0.0223 - https://www.kaggle.com/lunapandachan/h-m-trending-products-weekly-add-test/notebook
* LB: 0.0225 - Customer Age + Who Bought This Frequently Buy This
* LB: 0.0223 - Collaborative Filtering
* LB: 0.0221 - Customer Average Price


In [ ]:
sub0 = pd.read_csv('./last_purchase_colors.csv').sort_values('customer_id').reset_index(drop=True) 
sub1 = pd.read_csv('./submission_colab_merge.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('./trending.csv').sort_values('customer_id').reset_index(drop=True)

#These notebooks are produced from running the respective notebooks submitted
sub3 = pd.read_csv('./customer_age_who_bought_this_also_bought_this.csv').sort_values('customer_id').reset_index(drop=True)
sub4 = pd.read_csv('./collaborative_filtering.csv').sort_values('customer_id').reset_index(drop=True)
sub5 = pd.read_csv('./avg_price_wise_grouping.csv').sort_values('customer_id').reset_index(drop=True)


In [ ]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction0'] = sub0['prediction0'].astype(str)
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
sub0['prediction4'] = sub4['prediction']
sub0['prediction5'] = sub5['prediction']


del sub1, sub2, sub3, sub4, sub5
gc.collect()
sub0.head()

In [ ]:
def cust_blend(dt, W = [1,1,1,1,1,1]):

    #Create a list of all model predictions
    REC = []

    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    REC.append(dt['prediction4'].split())
    REC.append(dt['prediction5'].split())


    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])

sub0['prediction'] = sub0.apply(cust_blend, W = [1.20, 0.88, 0.08, 0.07, 0.05, 0.05], axis=1)

sub0.head()

In [ ]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
del sub0['prediction3']
del sub0['prediction4']
del sub0['prediction5']

gc.collect()

# Make a submission

In [ ]:
sub0.to_csv('submission.csv', index=False)